In [1]:
import sys # aby importy z dołu działały
sys.path.append(r"C:\Users\piotr\OneDrive\____Moje-MOJE\MyProjects_4Fun\projects\World of Warcraft\python-etl")

In [2]:
from sqlalchemy import text, bindparam
from moduly.db_core import utworz_engine_do_db, _czy_duplikat
from moduly.services_persist_wynik import przefiltruj_dane_misji, zapisz_misje_dialogi_ai_do_db
from moduly.ai_gemini import zaladuj_api_i_klienta, instrukcja_tlumacz, instrukcja_redaktor
from moduly.utils import sklej_warunki_w_WHERE
from sqlalchemy.exc import IntegrityError
from scraper_wiki_main import parsuj_misje_z_url
from google import genai
import json
import zlib
import base64
from bs4 import BeautifulSoup
import pandas as pd
from pprint import pprint

In [3]:
# chce stworzyc proces, w ktorym bede mogl zatwierdzac tlumaczenia
# potrzebuje do tego misje w podziale na grupy:
#    a. oryinal,
#    b. przetlumaczone,
#    c. zredagowane

# oraz potrzebuje do tego wypowiedzi w podziale na grupy:
#    a. oryinal,
#    b. przetlumaczone,
#    c. zredagowane

# oraz tlumaczenia NPCow, Slow kluczowych
# oraz sprawdzic kazde slowo w misji/wypowiedzi czy wystepuje w Slowach kluczowych (cross check)

# calosc potrzebuje wyrzucic do excela w takim ukladzie:
#    a. 

In [4]:
silnik = utworz_engine_do_db()

In [5]:
def stworz_excele_do_zatwaierdzenia_tlumaczen(silnik, kraina = None, fabula = None, dodatek = None):

    warunki_sql = sklej_warunki_w_WHERE(kraina, fabula, dodatek)

    q_select_misje = text(f"""
        SELECT m.MISJA_ID_MOJE_PK
        FROM dbo.MISJE AS m
        WHERE 1=1
        AND m.MISJA_ID_MOJE_PK <> 123456789
                        
        {warunki_sql}

        ORDER BY m.MISJA_ID_MOJE_PK ASC
        ;
    """)

    q_select_tytul = text(f"""
        SELECT MISJA_ID_MOJE_PK, MISJA_TYTUL_EN, MISJA_TYTUL_PL, NAZWA_LINII_FABULARNEJ_EN, NAZWA_LINII_FABULARNEJ_PL
        FROM dbo.MISJE
        WHERE MISJA_ID_MOJE_PK IN :misje_id
    ;
    """).bindparams(bindparam("misje_id", expanding=True))

    parametry = {
        "kraina_en": kraina, 
        "fabula_en": fabula, 
        "dodatek_en": dodatek
    }

    with silnik.connect() as conn:
        misje = conn.execute(q_select_misje, parametry).scalars().all()
        tytuly = conn.execute(q_select_tytul, {"misje_id": misje}).mappings().all()
        return tytuly


In [8]:
tytuly = stworz_excele_do_zatwaierdzenia_tlumaczen(silnik, kraina="Exile's Reach")
df_tytuly = pd.DataFrame(tytuly)
df_tytuly

,MISJA_ID_MOJE_PK,MISJA_TYTUL_EN,MISJA_TYTUL_PL,NAZWA_LINII_FABULARNEJ_EN,NAZWA_LINII_FABULARNEJ_PL
0,2,A Druid's Form,Druidzka postać,A Dark Ritual,None
1,3,A Druid's Form,Druidzka postać,A Dark Ritual,None
2,5,A Hunter's Trap,Pułapka łowcy,A Dark Ritual,None
3,6,A Hunter's Trap,Pułapka Łowcy,A Dark Ritual,None
4,8,A Mage's Knowledge,Wiedza maga,A Dark Ritual,None
...,...,...,...,...,...
96,147,Warming Up,Rozgrzewka,The Missing Expedition,None
97,150,Westward Bound,Na zachód,A Dark Ritual,None
98,151,Westward Bound,Na zachód,A Dark Ritual,None
99,156,Who Lurks in the Pit,Kto czai się w jamie,A Dark Ritual,None
